Import Libraries

In [124]:
import pandas as pd
import numpy as np
import os

### Warnings
import warnings

### Text Preprocessing and Natural Language Processing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import nltk
import re
import spacy
from wordcloud import WordCloud

# For Fuzzy matching techniques
from rapidfuzz.process import extractOne

# For parallel processing
from concurrent.futures import ThreadPoolExecutor


Import TMDB Dataset

In [125]:
pd.set_option('display.max_columns', None)

# Load in TMDB dataset
tmdb_df = pd.read_csv('TMDB_movie_dataset_v11.csv') 

# Drop Duplicates
tmdb_df = tmdb_df.drop_duplicates()

Creation of primary key in TMDB dataset (In order to join with the movielens dataset)

In [126]:
# Cleaning up the release date column to coerce problematic values to NaT, and ensure dates are in proper format
tmdb_df['release_date'] = pd.to_datetime(tmdb_df['release_date'], errors='coerce')

# Creating new "year" (string) column and extracting year from the "release_date" field
# Changed .replace('nan', '') to .replace('<NA>', '')
tmdb_df['year'] = tmdb_df['release_date'].dt.year.astype('Int64').astype(str).replace('<NA>', '')

# Clean up movie titles to remove leading and trailing whitespace if any
tmdb_df['title'] = tmdb_df['title'].str.strip()

# Create a new column called title_year, combining title and year. If year is available, then create a new field combining title and year
tmdb_df['title_year'] = tmdb_df.apply(lambda x: f"{x['title']} ({x['year']})" if x['year'] else x['title'], axis=1).str.strip()

Import MovieLens Datasets

In [127]:
# Load MovieLens Movies dataset
df_movies = pd.read_csv('ml-1m/movies.csv', sep='::', engine='python', 
                        encoding='ISO-8859-1', header=None, names=['MovieID', 'Title', 'Genres'])

# Clean up movie titles to remove leading and trailing whitespace
df_movies['Title'] = df_movies['Title'].str.strip()

# Load MovieLens Ratings dataset
df_ratings = pd.read_csv('ml-1m/ratings.csv', sep='::', engine='python', 
                         encoding='ISO-8859-1', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'])

# Load MovieLens Users dataset
df_users = pd.read_csv('ml-1m/users.csv', sep='::', engine='python', 
                       encoding='ISO-8859-1', header=None, names=['UserID', 'Gender', 'Age', 'Occupation','ZipCode'])

In [128]:
# Merge MovieLens Dataset into one dataframe (from three)
df_ml_combined = pd.merge(pd.merge(df_ratings, df_movies, on='MovieID'), df_users, on='UserID')

Truncate TMDB Dataset to show movies in Year 2000 and before only

In [129]:
# Filter out TMDB dataset for movies released in year 2000 and before
truncated_tmdb_df = tmdb_df[tmdb_df['release_date'].dt.year.astype('Int64') <= 2000]

Handle duplicates in the TMDB dateset involving the key 'title_year'

In [130]:
# Extract out rows from TMDB dataset with duplicate keys as they are non-unique
duplicates_tmdb = truncated_tmdb_df[truncated_tmdb_df['title_year'].duplicated(keep=False)]

# Get unique title-year keys from duplicates
unique_duplicated_title_years = duplicates_tmdb['title_year'].unique()

# Filter Combined MovieLens dataset to exclude rows with the duplicated keys from the analysis
filtered_df_ml_combined = df_ml_combined[~df_ml_combined['Title'].isin(unique_duplicated_title_years)]

Fixing Articles which appear at the end of movie titles in the filtered_df_ml_combined dataset

In [131]:
def format_title(title):
    # Split the title into words
    words = title.split()

    # Check if the last word is "A", "An", or "The"
    if len(words) >= 3 and words[-2].lower() in ['a', 'an', 'the'] and words[-3][-1] == ",":
        # Move the article to the beginning of the title
        formatted_title = f"{words[-2]} {' '.join(words[:-3])} {words[-3][:-1]} {words[-1]}"
    else:
        # Keep the title unchanged
        formatted_title = title

    return formatted_title

In [132]:
# Adjusting Movie Titles in correct sequence in the movielens dataset
filtered_df_ml_combined["Title"] = filtered_df_ml_combined["Title"].apply(format_title)

Fixing Movie Titles: Removing extra brackets --> as it is causing discrepancies in matching <br><br>
Example: "Shall We Dance? (Shall We Dansu?) (1996)" --> "Shall We Dance? (1996)"

In [133]:
# Function to remove inner parentheses
def remove_inner_parentheses(text):
  if text.count('(') >= 2:
      return text[:text.find('(')] + text[text.find(')')+1:].strip()
  else:
    return text

# Replace titles with two sets of parentheses ; Applying function twice as some titles have more than 2 brackets
filtered_df_ml_combined["Title"] = filtered_df_ml_combined['Title'].apply(remove_inner_parentheses).apply(remove_inner_parentheses)

Adding new columns to extract movie name and year seperately for movielens dataset

In [134]:
# Create new `Movie_Title` column and removing the year from the `Title` column
filtered_df_ml_combined["Movie_Title"] = filtered_df_ml_combined["Title"].str[:-7].str.strip()

# Create new `Movie_Year` column
filtered_df_ml_combined["Movie_Year"] = filtered_df_ml_combined["Title"].str[-5:-1].str.strip().astype(int)

In [135]:
# Stripping again to prepare for merge
filtered_df_ml_combined["Title"] = filtered_df_ml_combined["Title"].str.strip()

In [136]:
# Removing intermediate white spaces
filtered_df_ml_combined["Title"] = filtered_df_ml_combined["Title"].str.replace("\s+", " ", regex=True)
truncated_tmdb_df["title_year"] = truncated_tmdb_df["title_year"].str.replace("\s+", " ", regex=True)

Combining both MovieLens and TMDB datasets (First iteration)

In [137]:
# Merge the 2 datasets using left-join (This will fix the `article` problem also) 
merged_df = pd.merge(filtered_df_ml_combined, truncated_tmdb_df, left_on='Title', right_on='title_year', how='left') 

Fixing discrepancies in movie release year

In [138]:
# Extracting unsucessful merges and put in same format as filtered_df_ml_combined
unsuccessful_merge = merged_df[merged_df['title_year'].isnull()].iloc[:,:12]

#Retain original index
unsuccessful_merge["original_index"] = unsuccessful_merge.index

Combining both MovieLens and TMDB datasets (Second iteration) --> Fix Movie Release Year error in MovieLens dataset

In [139]:
# Stripping TMDB title column
truncated_tmdb_df["title"] = truncated_tmdb_df["title"].str.strip()

# Removing intermediate white spaces
filtered_df_ml_combined["Movie_Title"] = filtered_df_ml_combined["Movie_Title"].str.replace("\s+", " ", regex=True)
truncated_tmdb_df["title"] = truncated_tmdb_df["title"].str.replace("\s+", " ", regex=True)

In [140]:
# Merge the 2 datasets using left-join (second iteration) using movie-titles as keys 
temp_merged_df = pd.merge(unsuccessful_merge, truncated_tmdb_df, left_on='Movie_Title', right_on='title', how='left') 

# Only keep those successful merges
temp_merged_df = temp_merged_df[temp_merged_df["title"].notnull()]

# Convert year to integer for comparison
temp_merged_df["year"] = temp_merged_df["year"].astype(int)

# Create a new column and check if the movie years have a maximum absolute 1 year difference (To handle computational errors in the movielens dataset)
temp_merged_df['one_year_difference'] = abs(temp_merged_df['Movie_Year'] - temp_merged_df['year']) <= 1

# Only keep those rows where 1 year difference is TRUE
temp_merged_df = temp_merged_df[temp_merged_df['one_year_difference'] == True]

# Drop the 1 year difference column now
temp_merged_df.drop(columns=["one_year_difference"], inplace=True)

# Update Indexes to correspond to original merged_df
temp_merged_df.set_index("original_index", inplace= True)

In [141]:
# Update merged_df dataframe with these new values 
merged_df = merged_df.combine_first(temp_merged_df)

Fuzzy Matching Techniques for string matching (Non Exact Match)  --> Third Iteration

In [19]:
# Filter out rows where the merge was unsuccessful, and put in same format as filtered_df_ml_combined
unmerged_v2 = merged_df[merged_df['title_year'].isnull()].iloc[:, :12]

# Keep original index from merged_df
unmerged_v2["original_index"] = unmerged_v2.index

In [40]:
# Reference list of movie names from TMDB dataset for comparison
choices = truncated_tmdb_df['title_year'].unique() 

# Inner Function to find best match for each string (in-place modification)
def find_best_match_parallel(query_strings, choices, dataset, threshold=85):
    for index, row in query_strings.iterrows():
        query_title = row['Title']
        best_match = extractOne(query_title, choices, score_cutoff=threshold)
        if best_match and best_match[1] > threshold:
            dataset.at[index, "Title"] = best_match[0]

# Main function with multithreading
def extract_values(batch_size, dataset):
    batches = [dataset.iloc[i:i+batch_size] for i in range(0, len(dataset), batch_size)]

    with ThreadPoolExecutor() as executor:  # Use ThreadPoolExecutor for multithreading
        executor.map(find_best_match_parallel, batches, [choices]*len(batches), [dataset]*len(batches))

<h1> HEAVY COMPUTATIONAL PROCESSING HERE </h1>

Batch 1

In [53]:
# Batch 1 (First 20000)
unmerged_v2_batch1 = unmerged_v2.iloc[:20000,:]

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,ZipCode,Movie_Title,Movie_Year,original_index
44,1,260,4,978300760,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,F,1,10,48067,Star Wars: Episode IV - A New Hope,1977,44
64,2,1210,4,978298151,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War,M,56,16,70072,Star Wars: Episode VI - Return of the Jedi,1983,64
65,2,1792,3,978299941,U.S. Marshalls (1998),Action|Thriller,M,56,16,70072,U.S. Marshalls,1998,65
99,2,1690,3,978300051,Alien: Resurrection (1997),Action|Horror|Sci-Fi,M,56,16,70072,Alien: Resurrection,1997,99
104,2,1873,4,978298542,"Misérables, Les (1998)",Drama,M,56,16,70072,"Misérables, Les",1998,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...
281510,1695,260,5,974707446,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,M,25,1,97206,Star Wars: Episode IV - A New Hope,1977,281510
281532,1695,1196,5,974707446,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War,M,25,1,97206,Star Wars: Episode V - The Empire Strikes Back,1980,281532
281543,1695,1374,5,974707603,Star Trek: The Wrath of Khan (1982),Action|Adventure|Sci-Fi,M,25,1,97206,Star Trek: The Wrath of Khan,1982,281543
281569,1695,2700,4,974708688,"South Park: Bigger, Longer and Uncut (1999)",Animation|Comedy,M,25,1,97206,"South Park: Bigger, Longer and Uncut",1999,281569


In [54]:
# Main function for Batch 1
extract_values(1000, unmerged_v2_batch1)

Batch 2

In [63]:
# Batch 2 (Next 20000)
unmerged_v2_batch2 = unmerged_v2.iloc[20000:40000,:]

In [65]:
# Main function for Batch 2
extract_values(1000, unmerged_v2_batch2)

In [66]:
# Export to Excel
unmerged_v2_batch2.to_excel("Batch 2.xlsx")

Combine Batch 1 and 2 together

In [145]:
batch1_2 = pd.concat([unmerged_v2_batch1,unmerged_v2_batch2])

# Merge (4th iteration)
temp2_merged = pd.merge(batch1_2, truncated_tmdb_df, left_on='Title', right_on='title_year', how='left') 

# Update Indexes to correspond to original merged_df
temp2_merged.set_index("original_index", inplace= True)

In [147]:
# Update merged_df dataframe with these new values 
merged_df = merged_df.combine_first(temp2_merged)

Batch 3

In [84]:
# Batch 3 (Next 20000)
unmerged_v2_batch3 = unmerged_v2.iloc[40000:60000,:]

In [85]:
# Main function for Batch 3
extract_values(1000, unmerged_v2_batch3)

In [86]:
# Export to Excel
unmerged_v2_batch3.to_excel("Batch 3.xlsx")

Batch 4

In [87]:
# Batch 4 
unmerged_v2_batch4 = unmerged_v2.iloc[60000:,:]

In [88]:
# Main function for Batch 4
extract_values(1000, unmerged_v2_batch4)

In [89]:
# Export to Excel
unmerged_v2_batch4.to_excel("Batch 4.xlsx")

Combine Batch 3 & 4

In [149]:
batch3_4 = pd.concat([unmerged_v2_batch3,unmerged_v2_batch4])

# Merge (5th iteration)
temp3_merged = pd.merge(batch3_4, truncated_tmdb_df, left_on='Title', right_on='title_year', how='left') 

# Update Indexes to correspond to original merged_df
temp3_merged.set_index("original_index", inplace= True)

In [150]:
# Update merged_df dataframe again
merged_df = merged_df.combine_first(temp3_merged)

Analyze Unmerged Cells

In [155]:
# Filter out rows where the merge was unsuccessful, and put in same format as filtered_df_ml_combined
unmerged_v3 = merged_df[merged_df["title_year"].isnull()].iloc[:, :12]

Re-running similarity checks for unmerged cells (which somehow did not work on first run)

In [157]:
choices = truncated_tmdb_df['title_year'].unique() 

for index, row in unmerged_v3.iterrows():
    query_title = row['Title']
    best_match = extractOne(query_title, choices, score_cutoff=85)
    if best_match and best_match[1] >= 85:
        unmerged_v3.at[index, "Title"] = best_match[0]

In [164]:
# Set indexes before merging to retain its values
unmerged_v3["original_index"] = unmerged_v3.index

# Merge (6th iteration)
temp4_merged = pd.merge(unmerged_v3, truncated_tmdb_df, left_on='Title', right_on='title_year', how='left') 

# Update Indexes to correspond to original merged_df
temp4_merged.set_index("original_index", inplace= True)

In [166]:
# Update merged_df dataframe again [FINAL ONE]
merged_df = merged_df.combine_first(temp4_merged)

Check whether there are any more unsuccessful merges

In [170]:
merged_df["title_year"].isnull().sum()

0

Export file to csv

In [ ]:
merged_df.to_csv("Merged_df.csv")